In [2]:
import requests
import pandas as pd
from typing import Text
from flask import Flask, render_template
import pandas as pd
import requests
from datetime import timedelta, datetime, date
import time



In [12]:
leagueID = "888655"

def getTeamList():
    try:
        url2 = 'https://fantasy.premierleague.com/api/leagues-classic/' + \
            leagueID + '/standings/'
        r2 = requests.get(url2)
        json2 = r2.json()
        standings_df = pd.DataFrame(json2['standings'])
        league_df = pd.DataFrame(standings_df['results'].values.tolist())
        return league_df[['entry', 'player_name']]
    except:
        return None

def getNewEntries():
    url = 'https://fantasy.premierleague.com/api/leagues-classic/' + \
        leagueID + '/standings/'
    r = requests.get(url).json()
    return r['new_entries']['results']



# url = 'https://fantasy.premierleague.com/api/leagues-classic/' + \
#             leagueID + '/standings/'
# r = requests.get(url).json()
# if not r['standings']['results']:
#     print(getNewEntries())
    
getTeamList()

[{'entry': 4960414, 'entry_name': 'Taperpengermedglede', 'joined_time': '2022-08-01T18:47:42.149413Z', 'player_first_name': 'Hogne', 'player_last_name': 'Vestvik'}, {'entry': 1160728, 'entry_name': 'TheHoff', 'joined_time': '2022-08-01T11:16:13.722467Z', 'player_first_name': 'Magnus', 'player_last_name': 'Rishaug'}, {'entry': 4609794, 'entry_name': 'Winningpool', 'joined_time': '2022-07-31T13:12:33.535074Z', 'player_first_name': 'Oda', 'player_last_name': 'Haugane'}, {'entry': 2891568, 'entry_name': 'Haalandaise sause', 'joined_time': '2022-07-29T11:19:15.448699Z', 'player_first_name': 'Christiane', 'player_last_name': 'Kahrs'}, {'entry': 57182, 'entry_name': 'Nordens PSG', 'joined_time': '2022-07-29T11:19:15.374162Z', 'player_first_name': 'Simen', 'player_last_name': 'Madsen'}, {'entry': 2890241, 'entry_name': 'midlertidig', 'joined_time': '2022-07-29T11:19:15.297872Z', 'player_first_name': 'Tor Aimar', 'player_last_name': 'Carlsen'}, {'entry': 2581160, 'entry_name': 'Bergartene', 'jo

In [18]:
thisGw = 18
gwStart = 17

def getLastRoundPoints (teamID):
     url = 'https://fantasy.premierleague.com/api/entry/' + str(teamID) + '/history/'
     r = requests.get(url).json()['current']
     LastRoundPoints = r[thisGw - 2]['total_points'] - r[17 - 2]['total_points']
     return LastRoundPoints

def genereateLastPlacements():
     url = 'https://fantasy.premierleague.com/api/leagues-classic/173312/standings/'
     r = requests.get(url).json()['standings']['results']
     lastStandings = []
     for manager in r:
          lastStandings.append((getLastRoundPoints(manager['entry']), manager['entry']))
     lastStandings.sort(key=lambda y:y[0], reverse=True)
     
     return lastStandings

genereateLastPlacements()

[(75, 1113962),
 (71, 1942180),
 (69, 1349385),
 (66, 2403760),
 (60, 4312501),
 (60, 2873246),
 (59, 6165479),
 (57, 955967),
 (47, 3414480),
 (47, 2554334),
 (38, 956051)]

In [16]:
def genereateLastPlacements():
    url = 'https://fantasy.premierleague.com/api/leagues-classic/173312/standings/'
    r = requests.get(url).json()['standings']['results']
    lastStandings = []
    for manager in r:
        lastStandings.append((getLastRoundPoints(manager['entry']), manager['entry']))
    lastStandings.sort(key=lambda y:y[0], reverse=True)
    
    entrys = []
    for x, y in lastStandings:
        entrys.append(y)
    return entrys

In [6]:
genereateLastPlacements().index(956051)



2

In [57]:
[x for x, y in enumerate(genereateLastPlacements()) if y[1] == 956051]

[2]

In [17]:

url = 'https://fantasy.premierleague.com/api/entry/956051/history/'
r = requests.get(url).json()
avg:int = (r['past'][-1]['rank'] + r['past'][-2]['rank'] + r['past'][-3]['rank']) / 3
avg

624609.0

In [214]:
def getPlayerInfo(playerId): 
    url2 = 'https://fantasy.premierleague.com/api/event/' + str(thisGw) + '/live/'
    liveInfo = requests.get(url2).json()['elements']
    liveInfo = liveInfo[playerId-1]['explain']
    playerInfo = []
    for stats in liveInfo:
        for stat in stats['stats']:
            playerInfo.append(stat)

    df = pd.DataFrame(playerInfo)
    test = []
    visited = []
    for i in range(len(df)):
        tempIdentifier = df.at[i, 'identifier']
        if tempIdentifier not in visited:
            tempValue = df.loc[df['identifier'] == tempIdentifier, 'value'].sum()
            test.append({
                'identifier' : tempIdentifier,
                'value' : tempValue
            })
            visited.append(tempIdentifier)
    return test


In [216]:
getPlayerInfo(1)

[{'identifier': 'minutes', 'value': 0}]

In [190]:
import collections, functools, operator

In [192]:
playerInfo

[{'identifier': 'minutes', 'points': 0, 'value': 0},
 {'identifier': 'minutes', 'points': 2, 'value': 90},
 {'identifier': 'goals_scored', 'points': 8, 'value': 2},
 {'identifier': 'bonus', 'points': 2, 'value': 2}]

In [207]:
playerInfo[0].values()

dict_values(['minutes', 0, 0])

In [135]:
df = pd.DataFrame(playerInfo)
df

,identifier,points,value
0,minutes,0,0
1,minutes,2,90
2,goals_scored,8,2
3,bonus,2,2


In [211]:
test = []
visited = []
for i in range(len(df)):
    tempIdentifier = df.at[i, 'identifier']
    if tempIdentifier not in visited:
        tempValue = df.loc[df['identifier'] == tempIdentifier, 'value'].sum()
        test.append({
            'identifier' : tempIdentifier,
            'value' : tempValue
        })
        visited.append(tempIdentifier)
test

[{'identifier': 'minutes', 'value': 90},
 {'identifier': 'goals_scored', 'value': 2},
 {'identifier': 'bonus', 'value': 2}]

In [134]:
for stats in liveInfo:
    for stat in stats['stats']:
        tempList = list(stat.keys())
         

In [182]:
test[0]['identifier']

'goals_scored'